In [1]:
import numpy as np # linear algebra
import pandas as pd


## Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler , StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


In [3]:
import warnings
warnings.filterwarnings("ignore")


## Load your dataset

In [4]:
data = pd.read_csv("../DataSet/powerconsumption.csv")


FileNotFoundError: [Errno 2] No such file or directory: '../DataSet/powerconsumption.csv'

### Look at Dataset

In [ ]:
data.sample(5)


#### Converting  'Datetime' column to datetime

In [ ]:
data['Datetime'] = pd.to_datetime(data['Datetime'], format='%m/%d/%Y %H:%M', dayfirst=True)


Set the 'Dates' column as the index

In [ ]:
data.set_index('Datetime', inplace=True)


In [ ]:
data['TotalPowerConsumption']= data['PowerConsumption_Zone1'] + data['PowerConsumption_Zone2'] + data['PowerConsumption_Zone3']


In [ ]:
data= data.drop(['PowerConsumption_Zone1','PowerConsumption_Zone2','PowerConsumption_Zone3'],axis=1)


## Checking for Missing values

In [ ]:
data.isnull().sum()


## Checking for Outliers

In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(data)
plt.title('Outliers Detection')
plt.show()


Some Outliers found. Lets remove them

In [ ]:
RS = RobustScaler()


In [ ]:
x = data.drop(['TotalPowerConsumption'],axis=1)
y = data['TotalPowerConsumption']


In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(x)
plt.show()


In [ ]:
scaled_x = RS.fit_transform(x)


In [ ]:
x= pd.DataFrame(scaled_x, columns=x.columns)


In [ ]:
plt.figure(figsize=(12,6))
sns.boxplot(x)
plt.show()


In [ ]:
sns.boxplot(x='TotalPowerConsumption',data=data)
plt.title('Outlier Detection in TotalPwerConsumption')
plt.show()


In [ ]:
Q1 = data['TotalPowerConsumption'].quantile(0.25)
Q3 = data['TotalPowerConsumption'].quantile(0.75)

IQR = Q3 - Q1

lowerbound = Q1 - (1.5 * IQR)
upperbound = Q3 + (1.5 * IQR)

data = data[(data.TotalPowerConsumption >= lowerbound) & (data.TotalPowerConsumption <= upperbound)]


In [ ]:
sns.boxplot(x='TotalPowerConsumption',data=data)
plt.title('Outlier Detection in TotalPowerConsumption')
plt.show()


## Checking and Fixing Data Distribution

In [ ]:
sns.displot(data['Temperature'], kde=True)
plt.title('Distribution Plot of Temperature')
plt.show()


In [ ]:
data['Temperature'].skew()


In [ ]:
from scipy.stats import boxcox
data['Temperature'], lambda_boxcox = boxcox(data['Temperature'])


In [ ]:
data['Temperature'].skew()


In [ ]:
sns.displot(data['Temperature'], kde=True)
plt.title('Distribution Plot of Temperature')
plt.show()


In [ ]:
sns.displot(data['Humidity'], kde=True) 
plt.title('Distribution Plot of Humidity')
plt.show()


In [ ]:
data['Humidity'].skew()


In [ ]:
from sklearn.preprocessing import PowerTransformer
pt = PowerTransformer(method='yeo-johnson')
data['Humidity'] = pt.fit_transform(data[['Humidity']])


In [ ]:
data['Humidity'].skew()


In [ ]:
sns.displot(data['Humidity'], kde=True) 
plt.title('Distribution Plot of Humidity')
plt.show()


In [ ]:
sns.displot(data['WindSpeed'], kde=True) 
plt.title('Distribution Plot of Windspeed')
plt.show()


In [ ]:
data['WindSpeed'].skew()


In [ ]:
from scipy.stats import boxcox
data['WindSpeed'], lambda_boxcox = boxcox(data['WindSpeed'])


In [ ]:
data['WindSpeed'].skew()


In [ ]:
sns.displot(data['WindSpeed'], kde=True) 
plt.title('Distribution Plot of Windspeed')
plt.show()


In [ ]:
sns.displot(data['GeneralDiffuseFlows'], kde=True) 
plt.title('Distribution Plot of General Diffuse Flow')
plt.show()


In [ ]:
data['GeneralDiffuseFlows'].skew()


In [ ]:
data['GeneralDiffuseFlows'], lambda_boxcox = boxcox(data['GeneralDiffuseFlows'])


In [ ]:
data['GeneralDiffuseFlows'].skew()


In [ ]:
sns.displot(data['GeneralDiffuseFlows'], kde=True) 
plt.title('Distribution Plot of General Diffuse Flow')
plt.show()


In [ ]:
sns.displot(data['DiffuseFlows'], kde=True) 
plt.title('Distribution Plot of Diffuse Flow')
plt.show()


In [ ]:
data['DiffuseFlows'].skew()


In [ ]:
data['DiffuseFlows'], lambda_boxcox = boxcox(data['DiffuseFlows'])


In [ ]:
data['DiffuseFlows'].skew()


In [ ]:
sns.displot(data['DiffuseFlows'], kde=True) 
plt.title('Distribution Plot of Diffuse Flow')
plt.show()


In [ ]:
sns.displot(data['TotalPowerConsumption'], kde=True) 
plt.title('Distribution Plot of TotalPowerConsumption')
plt.show()


In [ ]:
data['TotalPowerConsumption'].skew()


In [ ]:
data['TotalPowerConsumption'], lambda_boxcox = boxcox(data['TotalPowerConsumption'])


In [ ]:
data['TotalPowerConsumption'].skew()


In [ ]:
sns.displot(data['TotalPowerConsumption'], kde=True) 
plt.title('Distribution Plot of TotalPowerConsumption')
plt.show()


In [ ]:
sns.boxplot(data)
plt.show()


In [ ]:
x= data.drop(['TotalPowerConsumption'],axis=1)
y=data['TotalPowerConsumption']


## Scaling

In [ ]:
Scaler = StandardScaler()


In [ ]:
x_scaled = Scaler.fit_transform(x) 


In [ ]:
x_scaled


In [ ]:
x_scaled.shape


In [ ]:
y.shape


In [ ]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=42)


#### AdFuller Report

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(y_train)
print(f'ADF Statistic: {result[0]}')
print(f'p-value: {result[1]}')


## ARIMA

In [ ]:
import pmdarima as pm
auto_model = pm.auto_arima(y_train, seasonal=False, trace=True,error_action='ignore', suppress_warnings=True)


In [ ]:
auto_model.fit(y_train)


In [ ]:
arima_pred = auto_model.predict(n_periods=len(y_test))


In [ ]:
print(auto_model.summary())


In [ ]:
mse = mean_squared_error(y_test, arima_pred)
print(f'Mean Squared Error: {mse}')


## SRIMAX

In [ ]:
auto_model = pm.auto_arima(y_train,seasonal=True, m=24,trace=True, error_action='ignore', suppress_warnings=True)


In [ ]:
auto_model.fit(y_train)


In [ ]:
sarimax_pred = auto_model.predict(n_periods=len(y_test))


In [ ]:
print(auto_model.summary())


In [ ]:
mse = mean_squared_error(y_test, sarimax_pred)
print(f'Mean Squared Error (SARIMAX): {mse}')


In [ ]:
# Plotting results
plt.figure(figsize=(14, 7))
plt.plot(x_test.index, y_test, label='Actual', color='black')  # Actual values
plt.plot(x_test.index, arima_pred, label='ARIMA Forecast', color='blue')  # ARIMA forecast
plt.plot(x_test.index, sarimax_pred, label='SARIMAX Forecast', color='orange')  # SARIMAX forecast
plt.title('Forecast vs Actuals')
plt.xlabel('Date')
plt.ylabel('Power Consumption')
plt.legend()
plt.show()


In [ ]:
# Prepare the x-axis positions
x_positions = np.arange(len(x_test))

# Create the bar width
bar_width = 0.25

# Plotting results using bar charts
plt.figure(figsize=(14, 7))

# Actual values
plt.bar(x_positions, y_test, width=bar_width, label='Actual', color='black', alpha=0.6)

# ARIMA forecast
plt.bar(x_positions + bar_width, arima_pred, width=bar_width, label='ARIMA Forecast', color='blue', alpha=0.6)

# SARIMAX forecast
plt.bar(x_positions + 2 * bar_width, sarimax_pred, width=bar_width, label='SARIMAX Forecast', color='orange', alpha=0.6)

# Adding titles and labels
plt.title('Forecast vs Actuals')
plt.xlabel('Time Periods')
plt.ylabel('Power Consumption')
plt.xticks(x_positions + bar_width, x_test.index.date, rotation=45)  # Rotate date labels for better readability
plt.legend()
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels
plt.show()
